## Цель предобработки данных

Цель данного этапа — преобразовать текстовые отзывы в числовой формат,
пригодный для обучения нейросетевой модели на основе LSTM.

В ходе предобработки мы:
- очищаем текст от шума
- токенизируем отзывы
- строим словарь
- кодируем слова в индексы
- выравниваем длины последовательностей
- подготавливаем данные для PyTorch

In [31]:
from pathlib import Path
import pandas as pd
import re
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")
from collections import Counter
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import pickle
from pathlib import Path

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nurs\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nurs\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Загрузка данных


In [3]:
DATA_PATH = Path(r"C:\\Users\\nurs\\OneDrive\\Рабочий стол\\IMDB Dataset.csv")
df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
df.head(3)

Shape: (50000, 2)
Columns: ['review', 'sentiment']


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


Датасет содержит:
- review — текст отзыва
- sentiment — метка тональности (positive / negative)


## Преобразование целевой переменной


In [ ]:
df["label"] = df["sentiment"].map({"negative": 0, "positive": 1})
print(df["label"].value_counts())
df[["sentiment", "label"]].head()

label
1    25000
0    25000
Name: count, dtype: int64


,sentiment,label
0,positive,1
1,positive,1
2,positive,1
3,negative,0
4,positive,1


Для обучения модели целевая переменная переводится в числовой формат:
- 1 — положительный отзыв
- 0 — отрицательный отзыв

Датасет является полностью сбалансированным.

## Базовая очистка текста


In [7]:
def clean_text_basic(text: str):
    text = re.sub(r"<.*?>", " ", text)
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text
df["review_clean"] = df["review"].apply(clean_text_basic)
df[["review", "review_clean"]].head(2)

,review,review_clean
0,One of the other reviewers has mentioned that ...,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,a wonderful little production the filming tech...


На данном шаге:
- удаляются HTML-теги
- текст приводится к нижнему регистру
- нормализуются пробелы

Агрессивная очистка (удаление стоп-слов, лемматизация) не применяется,
так как она может ухудшить качество LSTM-моделей.

## Токенизация текста


In [15]:
def tokenize_text(text: str):
    return word_tokenize(text)
df["tokens"] = df["review_clean"].apply(tokenize_text)
df[["review_clean", "tokens"]].head(2)

,review_clean,tokens
0,one of the other reviewers has mentioned that ...,"[one, of, the, other, reviewers, has, mentione..."
1,a wonderful little production the filming tech...,"[a, wonderful, little, production, the, filmin..."


Токенизация выполняется с использованием библиотеки NLTK.
Каждый отзыв преобразуется в список слов (токенов),
что является оптимальным входом для LSTM.


## Построение словаря


In [19]:
VOCAB_SIZE = 20_000
PAD_TOKEN = "<PAD>"
UNK_TOKEN = "<UNK>"
all_tokens = []
for tokens in df["tokens"]:
    all_tokens.extend(tokens)
token_freqs = Counter(all_tokens)
most_common_tokens = token_freqs.most_common(VOCAB_SIZE - 2)
word2idx = {
    PAD_TOKEN: 0,
    UNK_TOKEN: 1
}

for idx, (word, _) in enumerate(most_common_tokens, start=2):
    word2idx[word] = idx
print("Размер словаря:", len(word2idx))

Размер словаря: 20000


Словарь строится на основе частоты слов:
- используется 20 000 наиболее частых слов
- <PAD> используется для padding
- <UNK> используется для неизвестных слов

## Кодирование текста


In [ ]:
def encode_tokens(tokens, word2idx):
    return [
        word2idx.get(token, word2idx["<UNK>"])
        for token in tokens
    ]

df["encoded"] = df["tokens"].apply(lambda x: encode_tokens(x, word2idx))
df[["tokens", "encoded"]].head(2)

,tokens,encoded
0,"[one, of, the, other, reviewers, has, mentione...","[29, 5, 2, 78, 2062, 47, 1063, 12, 101, 150, 4..."
1,"[a, wonderful, little, production, the, filmin...","[4, 395, 121, 355, 2, 1383, 2983, 7, 54, 17699..."


Каждое слово заменяется на соответствующий числовой индекс из словаря.
Слова, отсутствующие в словаре, кодируются как <UNK>.


## Padding и Truncation


In [22]:
MAX_LEN = 400
PAD_IDX = word2idx["<PAD>"]
def pad_truncate(sequence, max_len, pad_idx):
    if len(sequence) > max_len:
        return sequence[:max_len]
    else:
        return sequence + [pad_idx] * (max_len - len(sequence))
df["padded"] = df["encoded"].apply(
    lambda x: pad_truncate(x, MAX_LEN, PAD_IDX)
)
df["padded"].apply(len).value_counts().head()

padded
400    50000
Name: count, dtype: int64

Все последовательности приводятся к фиксированной длине 400:
- длинные отзывы усекаются
- короткие дополняются <PAD>

Это необходимо для батчевого обучения нейросети.


## Разделение данных


In [24]:
X = df["padded"].tolist()
y = df["label"].tolist()
X_train, X_temp, y_train, y_temp = train_test_split(
    X,
    y,
    test_size=0.30,
    stratify=y,
    random_state=42
)

In [25]:
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.50,
    stratify=y_temp,
    random_state=42
)

Используется разбиение:
- 70% — обучение
- 15% — валидация
- 15% — тест

Баланс классов сохраняется с помощью stratify.


In [26]:
print("Train size:", len(X_train))
print("Val size:", len(X_val))
print("Test size:", len(X_test))

Train size: 35000
Val size: 7500
Test size: 7500


In [28]:
class IMDBDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.long)
        y = torch.tensor(self.y[idx], dtype=torch.float)
        return x, y
train_dataset = IMDBDataset(X_train, y_train)
val_dataset   = IMDBDataset(X_val, y_val)
test_dataset  = IMDBDataset(X_test, y_test)
len(train_dataset), len(val_dataset), len(test_dataset)

(35000, 7500, 7500)

In [30]:
BATCH_SIZE = 64
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)
batch_X, batch_y = next(iter(train_loader))
batch_X.shape, batch_y.shape


(torch.Size([64, 400]), torch.Size([64]))

In [ ]:
idx2word = {idx: word for word, idx in word2idx.items()}

## Сохранение данных


In [ ]:
SAVE_DIR = Path("artifacts")
SAVE_DIR.mkdir(exist_ok=True)
with open(SAVE_DIR / "word2idx.pkl", "wb") as f:
    pickle.dump(word2idx, f)
with open(SAVE_DIR / "idx2word.pkl", "wb") as f:
    pickle.dump(idx2word, f)


In [35]:
torch.save(
    {
        "X_train": torch.tensor(X_train, dtype=torch.long),
        "y_train": torch.tensor(y_train, dtype=torch.float),
        "X_val":   torch.tensor(X_val, dtype=torch.long),
        "y_val":   torch.tensor(y_val, dtype=torch.float),
        "X_test":  torch.tensor(X_test, dtype=torch.long),
        "y_test":  torch.tensor(y_test, dtype=torch.float),
    },
    SAVE_DIR / "dataset.pt"
)

In [36]:
len(word2idx), word2idx["<PAD>"], word2idx["<UNK>"]

(20000, 0, 1)